# T3: Neural IBM1 (with collocation)

In [1]:
# first run a few imports:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from pprint import pprint
import pickle
from utils import smart_reader, bitext_reader
from vocabulary import OrderedCounter, Vocabulary 
from utils import iterate_minibatches, prepare_data

# Load Data

In [2]:
# the paths to our training and validation data, English side
train_e_path = 'data/training/hansards.36.2.e.gz'
train_f_path = 'data/training/hansards.36.2.f.gz'
dev_e_path = 'data/validation/dev.e.gz'
dev_f_path = 'data/validation/dev.f.gz'
dev_wa = 'data/validation/dev.wa.nonullalign'
test_e_path = 'data/test/test.e.gz'
test_f_path = 'data/test/test.f.gz'
test_wa = 'data/test/test.wa.nonullalign'

In [5]:
# Using only 1000 words will result in many UNKs, but
# it will make training a lot faster. 
# If you have a fast computer, a GPU, or a lot of time,
# try with 10000 instead.
max_tokens=1000

corpus_e = smart_reader(train_e_path)    
vocabulary_e = Vocabulary(corpus=corpus_e, max_tokens=max_tokens)
pickle.dump(vocabulary_e, open("vocabulary_e.pkl", mode="wb"))

corpus_f = smart_reader(train_f_path)    
vocabulary_f = Vocabulary(corpus=corpus_f, max_tokens=max_tokens)
pickle.dump(vocabulary_f, open("vocabulary_f.pkl", mode="wb"))

# Model and Train

In [15]:
from neuralibm1 import NeuralIBM1Model
from neuralibm1trainer import NeuralIBM1Trainer

# Collocation

In [18]:
tf.reset_default_graph()

with tf.Session() as sess:

    # some hyper-parameters
    # tweak them as you wish
    batch_size=16  # on CPU, use something much smaller e.g. 1-16
    max_length=30
    lr = 0.001
    lr_decay = 0.0  # set to 0.0 when using Adam optimizer (default)
    emb_dim = 64
    mlp_dim = 128

    # our model
    # change context to : "gate", "concat", or "col_discrete". "col_discrete is for T3"
    model = NeuralIBM1Model(
        x_vocabulary=vocabulary_e, y_vocabulary=vocabulary_f, 
        batch_size=batch_size, emb_dim=emb_dim, mlp_dim=mlp_dim, session=sess, context="col_discrete")

    # our trainer
    trainer = NeuralIBM1Trainer(
        model, train_e_path, train_f_path, 
        dev_e_path, dev_f_path, dev_wa,
        test_e_path, test_f_path, test_wa,
        num_epochs=10, batch_size=batch_size, 
        max_length=max_length, lr=lr, lr_decay=lr_decay, session=sess)

    # now first TF needs to initialize all the variables
    print("Initializing variables..")
    sess.run(tf.global_variables_initializer())

    # now we can start training!
    print("Training started..")
    results = trainer.train()
    dev_AERs, test_AERs, train_likelihoods, dev_likelihoods = results

Training with B=16 max_length=30 lr=0.001 lr_decay=0.0
Initializing variables..
Training started..
Shuffling training data
Iter   100 loss 68.553726 accuracy 0.16 lr 0.001000
Iter   200 loss 58.562141 accuracy 0.20 lr 0.001000
Iter   300 loss 51.571232 accuracy 0.18 lr 0.001000
Iter   400 loss 56.012936 accuracy 0.18 lr 0.001000
Iter   500 loss 64.196602 accuracy 0.17 lr 0.001000
Iter   600 loss 50.099098 accuracy 0.20 lr 0.001000
Iter   700 loss 45.803978 accuracy 0.20 lr 0.001000
Iter   800 loss 53.881210 accuracy 0.22 lr 0.001000
Iter   900 loss 64.321457 accuracy 0.17 lr 0.001000
Iter  1000 loss 51.318100 accuracy 0.21 lr 0.001000
Iter  1100 loss 58.482121 accuracy 0.19 lr 0.001000
Iter  1200 loss 39.769440 accuracy 0.22 lr 0.001000
Iter  1300 loss 51.398457 accuracy 0.13 lr 0.001000
Iter  1400 loss 49.927032 accuracy 0.19 lr 0.001000
Iter  1500 loss 48.800968 accuracy 0.21 lr 0.001000
Iter  1600 loss 48.834038 accuracy 0.18 lr 0.001000
Iter  1700 loss 64.203423 accuracy 0.16 lr 0.

KeyboardInterrupt: 

In [ ]:
# plotting
import matplotlib.pyplot as plt

handles = []
ax1 = plt.plot(range(1, len(dev_AERs)+1), dev_AERs, label='dev-set')
handles.extend(ax1)
ax2 = plt.plot(range(1, len(test_AERs)+1), test_AERs, label='test-set')
handles.extend(ax2)
plt.legend(handles=handles)
plt.xlabel('Epochs')
plt.ylabel('AER')
plt.show()
plt.clf()

handles = []
ax1 = plt.plot(range(1, len(train_likelihoods)+1), train_likelihoods, label='training-set')
handles.extend(ax1)
plt.legend(handles=handles)
plt.xlabel('Epochs')
plt.ylabel('log-likelihood')
plt.show()
plt.clf()

handles = []
ax1 = plt.plot(range(1, len(dev_likelihoods)+1), dev_likelihoods, label='dev-set')
handles.extend(ax1)
plt.legend(handles=handles)
plt.xlabel('Epochs')
plt.ylabel('log-likelihood')
plt.show()
plt.clf()